In [4]:
from keys import FTX_API_KEY, FTX_API_SECRET
from datetime import datetime
import time
import hmac
import pandas as pd
from requests import Request, Session, Response

class FtxClient:


    def __init__(self, api_key=None, api_secret=None, subaccount_name=None):
            self._session = Session()
            self._api_key = api_key
            self._api_secret = api_secret
            self._subaccount_name = subaccount_name
            self._ENDPOINT = 'https://ftx.com/api/'


    def sign_request(self, request: Request):
        ts = int(time.time() * 1000)
        request = Request('GET', self._ENDPOINT)
        prepared = request.prepare()
        print(prepared)
        signature_payload = f'{ts}{prepared.method}{prepared.path_url}'.encode()
        signature = hmac.new(self._api_secret.encode(), signature_payload, 'sha256').hexdigest()

        request.headers['FTX-KEY'] = self._api_key
        request.headers['FTX-SIGN'] = signature
        request.headers['FTX-TS'] = str(ts)

    def send_request(self, method: str, path: str, **kwargs):
        request = Request(method, self._ENDPOINT + path, **kwargs)
        self.sign_request(request)
        response = self._session.send(request.prepare())
        return self._process_response(response)

    def _process_response(self, response: Response):
        try:
            data = response.json()
        except ValueError:
            response.raise_for_status()
            raise
        else:
            if not data['success']:
                raise Exception(data['error'])
            return data['result']
    
    def get_all_trades(self, market: str, start_time: float = None, end_time: float = None):
        ids = set()
        limit = 100
        results = []
        while True:
            response = self._get(f'markets/{market}/trades', {
                'end_time': end_time,
                'start_time': start_time,
            })
            deduped_trades = [r for r in response if r['id'] not in ids]
            results.extend(deduped_trades)
            ids |= {r['id'] for r in deduped_trades}
            print(f'Adding {len(response)} trades with end time {end_time}')
            if len(response) == 0:
                break
            end_time = min(parse_datetime(t['time']) for t in response).timestamp()
            if len(response) < limit:
                break
        return results


In [9]:
import json

dt = datetime(2020, 1, 1).timestamp()
acc = FtxClient(api_key=FTX_API_KEY, api_secret=FTX_API_SECRET)
res = acc.send_request('GET', 'markets/BTC-PERP/candles?resolution=15&limit=10')
df = pd.DataFrame(res)
print(df)

<PreparedRequest [GET]>
     close     high      low     open                  startTime  \
0  56791.0  56792.0  56787.0  56788.0  2021-03-12T05:04:15+00:00   
1  56794.0  56794.0  56787.0  56791.0  2021-03-12T05:04:30+00:00   
2  56808.0  56808.0  56789.0  56794.0  2021-03-12T05:04:45+00:00   
3  56846.0  56848.0  56808.0  56808.0  2021-03-12T05:05:00+00:00   
4  56863.0  56864.0  56840.0  56848.0  2021-03-12T05:05:15+00:00   
5  56857.0  56912.0  56855.0  56863.0  2021-03-12T05:05:30+00:00   
6  56900.0  56905.0  56857.0  56857.0  2021-03-12T05:05:45+00:00   
7  56887.0  56907.0  56887.0  56900.0  2021-03-12T05:06:00+00:00   
8  56877.0  56887.0  56851.0  56887.0  2021-03-12T05:06:15+00:00   
9  56867.0  56877.0  56867.0  56877.0  2021-03-12T05:06:30+00:00   

           time        volume  
0  1.615525e+12  9.480976e+04  
1  1.615525e+12  1.227743e+04  
2  1.615525e+12  1.068108e+05  
3  1.615526e+12  9.232214e+04  
4  1.615526e+12  2.535592e+05  
5  1.615526e+12  1.181210e+06  
6  